In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import random_split, DataLoader
from cnn_utils import train, accuracy

In [2]:
seed = 123
dtype = torch.float32
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
torch.set_default_device(device)
torch.manual_seed(seed)
generator = torch.Generator(device=device).manual_seed(seed)

In [3]:
dataset = torchvision.datasets.MNIST(root = './cnn_dataset',
                                               train = True,
                                               transform = transforms.ToTensor(),
                                               download = True)
test_dataset = torchvision.datasets.MNIST(root = './cnn_dataset',
                                               train = False,
                                               transform = transforms.ToTensor(),
                                               download = True)

total_size = len(dataset)
train_size = int(total_size * 0.8)
val_size = int(total_size * 0.2)

train_dataset, val_dataset = random_split(dataset, [train_size, val_size], generator=generator)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, generator=generator)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True, generator=generator)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True, generator=generator)

In [4]:
class LeNet(nn.Module):

    def __init__(self, num_classes):
        super(LeNet, self).__init__()
        self.layers = nn.Sequential(
            nn.LazyConv2d(6, kernel_size=5, padding=2),
            nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.LazyConv2d(16, kernel_size=5),
            nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.LazyLinear(120),
            nn.Sigmoid(),
            nn.LazyLinear(84),
            nn.Sigmoid(),
            nn.LazyLinear(num_classes)
        )

    def forward(self, x):
        return self.layers(x)

In [5]:
output_size = 10
learning_rate = 1e-3

model = LeNet(output_size)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
config = {
    "epoch" : 20,
    "print_per_epoch": 2,
    "device": device
}

train(model, criterion, optimizer, train_loader, val_loader, config)
acc = accuracy(model, test_loader, device)

print('ACC {}'.format(acc))

LOSS train 0.24041746968651811 valid 0.20754111501574515
LOSS train 0.11815097906067967 valid 0.11709891218940417
LOSS train 0.08561533959023654 valid 0.08392027513186137
LOSS train 0.07073079738890131 valid 0.0779736869049569
LOSS train 0.06242267719112957 valid 0.07903549768651526
LOSS train 0.05849538954766467 valid 0.07341436232191821
LOSS train 0.054217027234689644 valid 0.0632495189147691
LOSS train 0.05231701295915991 valid 0.06130173058869938
LOSS train 0.051187940732731176 valid 0.0549211854878813
LOSS train 0.0476602442082949 valid 0.07479409708703558
ACC 0.981
